In this notebook I will plot all the longidutinal data for each individual separately

In [2]:
import src.data.breathe_data as br
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import src.data.helpers as dh

In [3]:
# df = br.build_O2_FEV1_FEF2575_df(meas_file=2, remove_nan=False)
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")


*** Building O2Sat, FEV1, FEF2575 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  48978
Number of FEV1 recordings: 41791
Number of FEF2575 recordings: 37068
Number of PEF recordings: 16755
Number of O2 Saturation recordings: 43930
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warni

In [6]:
# df.to_excel(f"{dh.get_path_to_main()}ExcelFiles/BR/BR_O2_FEV1_FEF2575_PEF_Nan.xlsx", index=False)

In [4]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-01-25,1.31,97.0,0.54,NaN,1.31,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,99.845492
1,101,2019-01-26,1.31,98.0,0.57,NaN,1.31,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,100.874827
2,101,2019-01-27,1.31,96.0,0.67,NaN,1.31,53,Male,173.0,3.610061,97.150104,36.287474,36.287474,98.816157
3,101,2019-01-28,1.30,96.0,0.69,NaN,1.31,53,Male,173.0,3.610061,97.150104,36.287474,36.010470,98.816157
4,101,2019-01-29,1.28,98.0,0.60,NaN,1.30,53,Male,173.0,3.610061,97.150104,36.010470,35.456463,100.874827


In [9]:
def plot_profile_for_ID(df_for_ID):
    df_for_ID = df_for_ID.reset_index()
    fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.04,)
    fig.add_trace(
        go.Scatter(
            y=df_for_ID["O2 Saturation"], x=df_for_ID["Date Recorded"], mode="markers"
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.ecFEV1, x=df_for_ID["Date Recorded"], mode="markers"),
        row=3,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.FEF2575, x=df_for_ID["Date Recorded"], mode="markers"),
        row=4,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df_for_ID.PEF / 60, x=df_for_ID["Date Recorded"], mode="markers"),
        row=5,
        col=1,
    )
    fig.update_yaxes(
        title="SpO2<br>(%)",
        row=1,
        col=1,
        range=[min(df["O2 Saturation"]), max(df["O2 Saturation"])],
    )
    fig.update_yaxes(
        title="FEV1<br>(L)",
        row=2,
        col=1,
        range=[min(df.FEV1) * 0.9, max(df.FEV1) * 1.1],
    )
    fig.update_yaxes(
        title="ecFEV1<br>(L)",
        row=3,
        col=1,
        range=[min(df.ecFEV1) * 0.9, max(df.ecFEV1) * 1.1],
    )
    fig.update_yaxes(
        title="FEF2575<br>(L/s)",
        row=4,
        col=1,
        range=[min(df.FEF2575) * 0.9, max(df.FEF2575) * 1.1],
    )
    fig.update_yaxes(
        title="PEF<br>(L/s)", row=5, col=1, range=[min(df.PEF) * 0.9, max(df.PEF) * 1.1]
    )
    fig.update_yaxes(nticks=10)
    fig.update_traces(marker=dict(size=2))
    title = f"Measures for ID {df_for_ID.ID[0]}"
    fig.update_layout(
        font=dict(size=8), showlegend=False, title=title, height=1000, width=1200
    )
    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/ID_profiles/{title}.pdf")
    return -1


df.groupby(by="ID").apply(plot_profile_for_ID)